In [94]:
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd
import plotly.express as px
import random

from dataset.util import *
%matplotlib inline

In [2]:
# bert-serving-start -model_dir data/uncased_L-24_H-1024_A-16 -num_worker=1 -max_seq_len=NONE -max_batch_size 64
from bert_serving.client import BertClient
bc = BertClient()

In [74]:
(conn, cur) = sql_connect()
articles = cur.execute('SELECT * FROM articles').fetchall()
texts = [a[4] for a in articles]
symbols = list(set([a[1] for a in articles]))
conn.close()
len(articles), articles[0]

(2995,
 (1,
  'AAPL',
  "Boomers and millennials both love Apple and Amazon, but here are the brands they don't agree on",
  '2020-03-07',
  "Millennials may not like labels in general, but they're as hooked on Amazon as boomers and Gen X.\n\n\nThat's according to the latest Brand Intimacy Survey by marketing and branding agency MBLM, which measures how emotionally connected Americans are to the brands they buy. Their scale is based on factors such as the positive feelings a user has for a company or label, and how much their values align, as well as the depth and intensity of the emotional connection in the brand relationship. And Visual Capitalist has made this data easily digestible by drawing up three charts that illustrate which age groups love which brands the most.\n\n\nAmazon AMZN, -1.19% was one of the few companies to be beloved across every age group, appearing in the top five for Gen Z/millennials, Gen X and baby boomers alike - and with boomers naming it their No. 1 brand 

In [21]:
(conn, cur) = sql_connect()
companies = cur.execute('SELECT * FROM companies').fetchall()
sym_to_comp = {}
for c in companies:
    sym_to_comp[c[1]] = c
conn.close()
len(companies), companies[0]

(81,
 (1,
  'AAPL',
  'Apple, Inc.',
  'Computers/Consumer Electronics',
  'Technology',
  'Apple, Inc. engages in the design, manufacture, and sale of smartphones, personal computers, tablets, wearables and accessories, and other variety of related services. It operates through the following geographical segments: Americas, Europe, Greater China, Japan, and Rest of Asia Pacific. The Americas segment includes North and South America. The Europe segment consists of European countries, as well as India, the Middle East, and Africa. The Greater China segment comprises of China, Hong Kong, and Taiwan. The Rest of Asia Pacific segment includes Australia and Asian countries. Its products and services include iPhone, Mac, iPad, AirPods, Apple TV, Apple Watch, Beats products, Apple Care, iCloud, digital content stores, streaming, and licensing services. The company was founded by Steven Paul Jobs, Ronald Gerald Wayne, and Stephen G. Wozniak on April 1, 1976 and is headquartered in Cupertino, C

In [6]:
X = bc.encode(texts)
X_embedded = TSNE(n_components=2).fit_transform(X)

In [68]:
df = pd.DataFrame({
    'x': X_embedded[:, 0], 'y': X_embedded[:, 1],
    'symb': [a[1] for a in articles],
    'sector': [sym_to_comp[a[1]][4] for a in articles],
    'is_tech': [sym_to_comp[a[1]][4]=='Technology' for a in articles],
    'virus': ['Yes' if ('covid' in a[4].lower() or 'corona' in a[4].lower()) else 'No' for a in articles],
    'headline': [a[2] for a in articles]
})
df.head()

,x,y,symb,sector,is_tech,virus,headline
0,27.632982,-16.429350,AAPL,Technology,True,No,Boomers and millennials both love Apple and Am...
1,9.401088,-14.792657,AAPL,Technology,True,Yes,Opinion: You've been diligently investing for ...
2,-56.899242,-16.786707,AAPL,Technology,True,Yes,"Coronavirus outbreak aside, China stocks just ..."
3,55.541245,-1.568942,AAPL,Technology,True,Yes,"Coronavirus update: 101,733 cases, 3,460 death..."
4,8.236340,-3.411009,AAPL,Technology,True,Yes,"What Apple, Microsoft, GE and other U.S. compa..."


In [70]:
fig = px.scatter(df, x="x", y="y", color="sector", hover_data=['sector', 'symb', 'headline'])
fig.show()

In [100]:
sym_to_idx = {s:symbols.index(s) for s in symbols}
sym_to_artid = {}
for s in symbols:
    arts = [a for a in articles if a[1] == s]
    art_idxs = [articles.index(a) for a in arts]
    random.shuffle(art_idxs)
    sym_to_artid[s] = art_idxs

A = []
AE = []
Y = []
for s in symbols:
    pos = sym_to_artid[s]
    n = len(pos)
    for i in range(n):
        A.append(sym_to_idx[s])
        AE.append(X[pos[i]])
        Y.append(1)
    for i in range(n):
        A.append(sym_to_idx[s])
        other_sym = s
        while other_sym == s:
            other_sym = random.choice(symbols)
        neg_ex = random.choice(sym_to_artid[other_sym])
        AE.append(X[neg_ex])
        Y.append(0)

A = np.array(A)
AE = np.array(AE)
Y = np.array(Y)
p = np.random.permutation(len(articles))
A = A[p]
AE = AE[p]
Y = Y[p]

In [81]:
from keras.layers import Input, Embedding, Dense, Dot, Reshape
from keras.models import Model

input_symbol = Input(shape=(1,))
input_art_emb = Input(shape=(1024,))

embedding_symbol = Embedding(input_dim=len(symbols), output_dim=512)(input_symbol)
embedding_symbol = Reshape((512,))(embedding_symbol)

embedding_art = Dense(512, activation='sigmoid')(input_art_emb)

merged = Dot(normalize=True, axes=1)([embedding_symbol, embedding_art])
merged = Reshape(target_shape=(1,))(merged)

out = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[input_symbol, input_art_emb], outputs=out)

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [140]:
hist = model.fit(x=[A, AE], y=Y, epochs=100, batch_size=16, validation_split=0.3)

Train on 2096 samples, validate on 899 samples
Epoch 1/100
2096/2096 [==============================] - 1s 688us/step - loss: 0.3729 - accuracy: 0.8845 - val_loss: 0.4485 - val_accuracy: 0.8198
Epoch 2/100
2096/2096 [==============================] - 1s 696us/step - loss: 0.3567 - accuracy: 0.8922 - val_loss: 0.4379 - val_accuracy: 0.8165
Epoch 3/100
2096/2096 [==============================] - 1s 638us/step - loss: 0.3403 - accuracy: 0.9065 - val_loss: 0.4289 - val_accuracy: 0.8187
Epoch 4/100
2096/2096 [==============================] - 1s 603us/step - loss: 0.3286 - accuracy: 0.9041 - val_loss: 0.4258 - val_accuracy: 0.8309
Epoch 5/100
2096/2096 [==============================] - 1s 665us/step - loss: 0.3206 - accuracy: 0.9117 - val_loss: 0.4187 - val_accuracy: 0.8276
Epoch 6/100
2096/2096 [==============================] - 1s 675us/step - loss: 0.3015 - accuracy: 0.9227 - val_loss: 0.4198 - val_accuracy: 0.8254
Epoch 7/100
2096/2096 [==============================] - 1s 660us/step 

2096/2096 [==============================] - 1s 621us/step - loss: 0.0483 - accuracy: 0.9943 - val_loss: 0.3951 - val_accuracy: 0.8487
Epoch 57/100
2096/2096 [==============================] - 1s 572us/step - loss: 0.0472 - accuracy: 0.9943 - val_loss: 0.3975 - val_accuracy: 0.8598
Epoch 58/100
2096/2096 [==============================] - 1s 600us/step - loss: 0.0446 - accuracy: 0.9948 - val_loss: 0.3844 - val_accuracy: 0.8576
Epoch 59/100
2096/2096 [==============================] - 1s 550us/step - loss: 0.0425 - accuracy: 0.9952 - val_loss: 0.4202 - val_accuracy: 0.8409
Epoch 60/100
2096/2096 [==============================] - 1s 581us/step - loss: 0.0431 - accuracy: 0.9943 - val_loss: 0.3962 - val_accuracy: 0.8576
Epoch 61/100
2096/2096 [==============================] - 1s 676us/step - loss: 0.0402 - accuracy: 0.9957 - val_loss: 0.3816 - val_accuracy: 0.8598
Epoch 62/100
2096/2096 [==============================] - 1s 648us/step - loss: 0.0384 - accuracy: 0.9957 - val_loss: 0.4009 

In [141]:
train_df = pd.DataFrame(hist.history).reset_index()
loss_df = pd.melt(train_df, 
    id_vars="index", value_vars=['val_loss', 'loss'], value_name="loss")
fig = px.line(loss_df, x="index", y="loss", color="variable")
fig.show()